### Install Requirements

In [ ]:
!pip install -r ./requirements.txt -q

In [1]:
from langchain.agents import (
    Tool,
    AgentExecutor,
    LLMSingleActionAgent,
    AgentOutputParser,
)
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

### Create Tools

In [9]:
from keys import apikey, aws_access_key_id, aws_secret_access_key
import os
from langchain.tools import BaseTool
from langchain.document_loaders import PyPDFLoader
import boto3
import json
from botocore.exceptions import ClientError

os.environ['OPENAI_API_KEY'] = apikey
#os.environ['aws_access_key_id'] = aws_access_key_id
#os.environ['aws_secret_access_key'] = aws_secret_access_key
aws_access_key_id=""
aws_secret_access_key=""

class LambdaRMQ(BaseTool):
    name = 'Lambda function for RMQ firewall rules'
    description = """Do NOT use this tool until you have received all input arguments from the user directly.
    
                    Use this tool to invoke a lambda function to create security group rules for only "RMQ" or "RabbitMQ".
                    
                    If the port is not for "RabbitMQ" or "RMQ", do not create a rule. Respond that you cannot open that port.
                    
                    Do not attempt to create a security group rule that is not for "RabbitMQ" or "RMQ" with this tool.
                    
                    ONLY accept action_input or input of "RMQ" or "RabbitMQ".
        
                    DO NOT accept action_input or input of anything else.
                    """

    def _run(self, get_log=False):

        """
        Do NOT use this tool until you have asked for the 'cidr' input argument.
        
        Use this tool to invoke a lambda function to create security group rules for only "RMQ" or "RabbitMQ"
        
        If the security group rule is not for "RabbitMQ" or "RMQ", do not create a rule. Respond that you cannot create that rule. 

        Only invoke the lambda function for input of "RMQ" or "RabbitMQ".
        
        ONLY accept action_input or input of "RMQ" or "RabbitMQ".
        
        DO NOT accept action_input or input of anything else. 

        :return: The response from the function invocation.
        """
        print("In the function")
        client = boto3.client('lambda', region_name='us-east-1', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)

        cidr_ranges = {'vdi': '10.10.64.0/24', 'surface': '10.10.65.0/24', 'development': '192.168.0.0/24'}
        cidr = ''
        if cidr == '':
            try:
                cidr = cidr_ranges.get(input(str('Please provide the group that will need access. (for example; vdi, surface, or development)')).lower())
            except KeyError:
                print('That is not a valid entry')
                _run()
        else:
            return 'Something went wrong. Please try again.'
   
        
        inputParams = {
        'cidr'    : cidr
        }
        
        response = client.invoke(
            FunctionName='darcy-agent-test-sg-rmq',
            Payload = json.dumps(inputParams)
            )
        return "I have created your security group rules for RMQ!"

    def _arun(self, sops):
        raise NotImplementedError("This tool does not support async")


class LambdaHTTPS(BaseTool):
    name = 'Lambda function for HTPPS firewall rules'
    description = """Use this tool to invoke a lambda function to create security group rules for only HTTPS.
                    
                    If the port is not for HTTPS, do not create a rule. Respond that you cannot open that port.
                    
                    Do not attempt to create a security group rule that is not for HTTPS with this tool.
                    
                    ONLY accept action_input or input of HTTPS.
        
                    DO NOT accept action_input or input of anything else.
                    
                    """

    def _run(self, function_name, get_log=False):

        """
        Use this tool to invoke a lambda function to create security group rules for only HTTPS
        
        If the port is not for HTTPS, do not create a rule. Respond that you cannot open that port.

        Only invoke the lambda function for input of HTTPS.
        
        ONLY accept action_input or input of HTTPS.
        
        DO NOT accept action_input or input of anything else.
        
        :return: The response from the function invocation.
        """
        
        client = boto3.client('lambda', region_name='us-east-1', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)

        response = client.invoke(
            FunctionName='darcy-agent-test-sg-https'
            )
        return "I have created your security group rules for HTTPS!"

    def _arun(self, sops):
        raise NotImplementedError("This tool does not support async")

### Create LLM, Memory, Agent, and Import Tools

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

llm = ChatOpenAI(
    openai_api_key=apikey,
    temperature=0,
    model_name='gpt-3.5-turbo'
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=2,
    return_messages=True
)

In [11]:
from langchain.agents import initialize_agent

tools = [LambdaRMQ(), LambdaHTTPS()]

agent = initialize_agent(
    agent='chat-conversational-react-description',
    #agent='zero-shot-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

docs = [
    Document(page_content=t.description, metadata={"index": i})
    for i, t in enumerate(tools)
]
vector_store = FAISS.from_documents(docs, OpenAIEmbeddings())
retriever = vector_store.as_retriever()


def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [tools[d.metadata["index"]] for d in docs]

In [ ]:
get_tools("Make me some coffee")

In [ ]:
# Set up the base template
template = """You are a world-class python coder. You have access to the following tools:

{tools}

You will be expected to modify the Python code from the most relevant tool provided to accomodate
what the Question is asking for.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! 

Question: {input}
{agent_scratchpad}"""

In [ ]:
from typing import Callable


# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

In [ ]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

In [ ]:
output_parser = CustomOutputParser()

In [ ]:
llm = OpenAI(temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)
tools = get_tools("what tools change security groups")
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [ ]:
agent_executor.run("What tool can perform RDP security group changes")

### Interact with the Agent

In [12]:
agent('Can you create security group rules for RabbitMQ?')

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new AgentExecutor chain...
{
    "action": "Lambda function for RMQ firewall rules",
    "action_input": "RMQ"
}In the function

Observation: I have created your security group rules for RMQ!
Thought:{
    "action": "Final Answer",
    "action_input": "The security group rules for RabbitMQ have been created successfully."
}

> Finished chain.


{'input': 'Can you create security group rules for RabbitMQ?',
 'chat_history': [],
 'output': 'The security group rules for RabbitMQ have been created successfully.'}

In [ ]:
#agent('Can you create a firewall rule for RDP?')